In [23]:
#pip install pymupdf 
#!pip install tqdm
#!pip install spacy 

In [16]:
pdf_path = r'C:\Users\Rohan\Desktop\Rohan\ML\RAG\human_nutrition.pdf'

In [20]:
import fitz
from tqdm.auto import tqdm 

def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip() 

    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    """
    Opens a PDF file, reads its text content page by page, and collects statistics.

    Parameters:
        pdf_path (str): The file path to the PDF document to be opened and read.

    Returns:
        list[dict]: A list of dictionaries, each containing the page number
        (adjusted), character count, word count, sentence count, token count, and the extracted text
        for each page.
    """
    doc = fitz.open(pdf_path)  # open a document
    pages_and_texts = [] # Empty list to hold page data
    for page_number, page in tqdm(enumerate(doc)):  # iterate the document pages
        text = page.get_text()  # get plain text encoded as UTF-8
        text = text_formatter(text)
        pages_and_texts.append({"page_number": page_number - 41,  # adjust page numbers since our PDF starts on page 42
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4,  # 1 token = ~4 chars, see: https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them
                                "text": text})
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[45]


0it [00:00, ?it/s]

{'page_number': 4,
 'page_char_count': 1181,
 'page_word_count': 202,
 'page_sentence_count_raw': 9,
 'page_token_count': 295.25,
 'text': 'Learning Objectives  By the end of this chapter, you will be able to:  •  Describe basic concepts in nutrition  •  Describe factors that affect your nutritional needs  •  Describe the importance of research and scientific  methods to understanding nutrition  What are Nutrients?  The foods we eat contain nutrients. Nutrients are substances  required by the body to perform its basic functions. Nutrients must  be obtained from our diet, since the human body does not  synthesize or produce them. Nutrients have one or more of three  basic functions: they provide energy, contribute to body structure,  and/or regulate chemical processes in the body. These basic  functions allow us to detect and respond to environmental  surroundings, move, excrete wastes, respire (breathe), grow, and  reproduce. There are six classes of nutrients required for the body  to

In [27]:
from spacy.lang.en import English # see https://spacy.io/usage for install instructions

nlp = English()

# Add a sentencizer pipeline, see https://spacy.io/api/sentencizer/ 
nlp.add_pipe("sentencizer")

for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)
    
    # Make sure all sentences are strings
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    
    # Count the sentences 
    item["page_sentence_count_spacy"] = len(item["sentences"])
pages_and_texts[45]

  0%|          | 0/1208 [00:00<?, ?it/s]

{'page_number': 4,
 'page_char_count': 1181,
 'page_word_count': 202,
 'page_sentence_count_raw': 9,
 'page_token_count': 295.25,
 'text': 'Learning Objectives  By the end of this chapter, you will be able to:  •  Describe basic concepts in nutrition  •  Describe factors that affect your nutritional needs  •  Describe the importance of research and scientific  methods to understanding nutrition  What are Nutrients?  The foods we eat contain nutrients. Nutrients are substances  required by the body to perform its basic functions. Nutrients must  be obtained from our diet, since the human body does not  synthesize or produce them. Nutrients have one or more of three  basic functions: they provide energy, contribute to body structure,  and/or regulate chemical processes in the body. These basic  functions allow us to detect and respond to environmental  surroundings, move, excrete wastes, respire (breathe), grow, and  reproduce. There are six classes of nutrients required for the body  to

In [30]:
# Define split size to turn groups of sentences into chunks
num_sentence_chunk_size = 10 

# Create a function that recursively splits a list into desired sizes
def split_list(input_list: list, 
               slice_size: int) -> list[list[str]]:
    """
    Splits the input_list into sublists of size slice_size (or as close as possible).

    For example, a list of 17 sentences would be split into two lists of [[10], [7]]
    """
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

# Loop through pages and texts and split sentences into chunks
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])
    
pages_and_texts[200]

  0%|          | 0/1208 [00:00<?, ?it/s]

{'page_number': 159,
 'page_char_count': 1762,
 'page_word_count': 303,
 'page_sentence_count_raw': 15,
 'page_token_count': 440.5,
 'text': 'thermoregulation is to balance heat gain with heat loss and body  water plays an important role in accomplishing this. Human life  is supported within a narrow range of temperature, with the  temperature set point of the body being 98.6°F (37°C). Too low or  too high of a temperature causes enzymes to stop functioning and  metabolism is halted. At 82.4°F (28°C) muscle failure occurs and  hypothermia sets in. At the opposite extreme of 111.2°F (44°C) the  central nervous system fails and death results. Water is good at  storing heat, an attribute referred to as heat capacity and thus helps  maintain the temperature set point of the body despite changes in  the surrounding environment.  There are several mechanisms in place that move body water  from place to place as a method to distribute heat in the body and  equalize body temperature (Figure 3.